In [1]:
%cd .. 

/home/teo/userdata/virtuosoNet


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from virtuoso.dataset import ScorePerformDataset, FeatureCollate, MultiplePerformSet, multi_collate, EmotionDataset
from virtuoso.parser import get_parser
from virtuoso import utils
from virtuoso.loss import LossCalculator

import numpy as np
import torch

In [4]:
parser = get_parser()
args = parser.parse_args(
    args=["--yml_path=ymls/han_measnote.yml",
          "--data_path=datasets/main_dataset_clamped/",
          "--emotion_data_path=datasets/emotion_dataset_clamped/",
          "--delta_loss=true",
          "--vel_balance_loss=true",
          "--device=cpu"]
)
args, net_params, configs = utils.handle_args(args)
device = 'cpu'

criterion = utils.make_criterion_func(args.loss_type)
loss_calculator = LossCalculator(criterion, args)

In [5]:
hier_type = ['is_hier', 'in_hier', 'hier_beat', 'hier_meas', 'meas_note']
curr_type = [x for x in hier_type if getattr(args, x)]


valid_set = ScorePerformDataset(args.data_path, 
                                type="valid", 
                                len_slice=args.len_valid_slice, 
                                len_graph_slice=args.len_graph_slice, 
                                graph_keys=args.graph_keys, 
                               )

In [6]:
valid_data_list = [{'path': item['score_path'], 'idx':i} for i, item in enumerate(valid_set.data)] 
valid_data_list.sort(key=lambda x:x['path'])
valid_data_list

[{'path': '/home/teo/userdata/datasets/chopin_cleaned/Bach/Fugue/bwv_865/musicxml_cleaned.musicxml',
  'idx': 8},
 {'path': '/home/teo/userdata/datasets/chopin_cleaned/Bach/Fugue/bwv_865/musicxml_cleaned.musicxml',
  'idx': 30},
 {'path': '/home/teo/userdata/datasets/chopin_cleaned/Bach/Fugue/bwv_865/musicxml_cleaned.musicxml',
  'idx': 87},
 {'path': '/home/teo/userdata/datasets/chopin_cleaned/Bach/Fugue/bwv_874/musicxml_cleaned.musicxml',
  'idx': 64},
 {'path': '/home/teo/userdata/datasets/chopin_cleaned/Bach/Fugue/bwv_874/musicxml_cleaned.musicxml',
  'idx': 77},
 {'path': '/home/teo/userdata/datasets/chopin_cleaned/Bach/Fugue/bwv_874/musicxml_cleaned.musicxml',
  'idx': 80},
 {'path': '/home/teo/userdata/datasets/chopin_cleaned/Bach/Prelude/bwv_865/musicxml_cleaned.musicxml',
  'idx': 13},
 {'path': '/home/teo/userdata/datasets/chopin_cleaned/Bach/Prelude/bwv_865/musicxml_cleaned.musicxml',
  'idx': 21},
 {'path': '/home/teo/userdata/datasets/chopin_cleaned/Bach/Prelude/bwv_865/mu

In [7]:
grouped_idx = []
prev_piece_path = valid_data_list[0]['path']
temp_groups = []
for item in valid_data_list:
  if item['path'] == prev_piece_path:
    temp_groups.append(item['idx'])
  else:
    prev_piece_path = item['path']
    if len(temp_groups) > 1:
      grouped_idx.append(temp_groups)
    temp_groups = []
grouped_idx

[[8, 30, 87],
 [77, 80],
 [21, 94],
 [52, 83],
 [44, 55],
 [76, 81],
 [19, 25, 34, 35, 38, 45, 78, 79],
 [16, 29, 59, 62, 65, 70, 72],
 [49, 54, 82],
 [17, 32, 39, 50, 60, 68, 85, 88],
 [5, 15, 27, 48, 63, 66, 93],
 [37, 57, 58, 74],
 [47, 69, 86],
 [23, 43, 46, 51, 56, 84, 91],
 [26, 73],
 [61, 92],
 [22, 28, 36, 41, 53, 89, 90]]

In [8]:
temp_groups

[14, 71]

In [9]:
# get perform features of same piece
valid_set.data[14]['output_data']
valid_set.data[71]['output_data']
valid_set.data[14]

{'input_data': array([[-2.3276691 , -0.08774704,  1.87368605, ...,  0.        ,
          0.6       ,  0.        ],
        [-0.6512695 , -0.43206026,  0.45568456, ...,  0.        ,
          0.6       ,  0.        ],
        [-0.51156953, -0.43206026, -0.60781655, ...,  0.        ,
          0.6       ,  0.        ],
        ...,
        [-0.6512695 , -0.08774704,  0.45568456, ...,  0.        ,
          0.6       ,  0.        ],
        [-1.4894693 , -0.08774704,  0.45568456, ...,  0.        ,
          0.6       ,  0.        ],
        [-2.3276691 , -0.08774704,  0.45568456, ...,  0.        ,
          0.6       ,  0.        ]]),
 'output_data': array([[-0.24673579, -0.86618847,  0.04202385, ...,  0.84888354,
         -1.10403234, -1.51603549],
        [ 0.95848965, -1.19855727,  0.04202385, ...,  0.84888354,
         -1.10403234, -1.51603549],
        [ 0.95848965, -0.97697807, -0.06807048, ...,  0.84888354,
         -1.10403234, -1.51603549],
        ...,
        [ 0.63161984,  1.

In [92]:
for group in grouped_idx:
  print(valid_set.data[group[0]]['score_path'])
  for i in range(len(group)-1):
    for j in range(i+1, len(group)):
      output_id = group[i]
      target_id = group[j]

      output = torch.Tensor(valid_set.data[output_id]['output_data']).unsqueeze(0)
      target = torch.Tensor(valid_set.data[target_id]['output_data']).unsqueeze(0)
      note_locations = {'beat': torch.LongTensor([valid_set.data[target_id]['note_location']['beat']])}
      align_matched = torch.LongTensor(valid_set.data[output_id]['align_matched']) * torch.LongTensor(valid_set.data[target_id]['align_matched'])
      align_matched = align_matched.unsqueeze(0).unsqueeze(-1)
      pedal_status = torch.Tensor(valid_set.data[target_id]['articulation_loss_weight'])
      pedal_status = pedal_status.unsqueeze(0).unsqueeze(-1)

      out, loss_dict = loss_calculator.cal_loss_by_term(output, target, note_locations, align_matched, pedal_status)
      print(loss_dict)
#       if out == 0:
#         print(output_id, target_id, valid_set.data[output_id]['perform_path'])

/home/teo/userdata/datasets/chopin_cleaned/Bach/Fugue/bwv_865/musicxml_cleaned.musicxml
{'tempo': 0.004100224003195763, 'vel': 0.7188635468482971, 'dev': 0.12144780904054642, 'articul': 1.6302827596664429, 'pedal': 3.1076056957244873, 'tempo_delta': 0.0016048089601099491, 'vel_balance': 0.3146619200706482}
{'tempo': 0.0042294785380363464, 'vel': 0.637650191783905, 'dev': 0.1961897313594818, 'articul': 1.2803213596343994, 'pedal': 0.4283192455768585, 'tempo_delta': 0.0016325950855389237, 'vel_balance': 0.279987633228302}
{'tempo': 0.003961730748414993, 'vel': 0.388516366481781, 'dev': 0.1662934124469757, 'articul': 0.9495317935943604, 'pedal': 3.064208984375, 'tempo_delta': 0.0017147139878943563, 'vel_balance': 0.17627350986003876}
/home/teo/userdata/datasets/chopin_cleaned/Bach/Fugue/bwv_874/musicxml_cleaned.musicxml
{'tempo': 0.014769017696380615, 'vel': 0.414255291223526, 'dev': 0.29226040840148926, 'articul': 0.5082284808158875, 'pedal': 1.1186972856521606, 'tempo_delta': 0.00149923

{'tempo': 0.039420727640390396, 'vel': 0.524475634098053, 'dev': 0.25634098052978516, 'articul': 0.4968227744102478, 'pedal': 1.2734218835830688, 'tempo_delta': 0.01085773017257452, 'vel_balance': 0.15799984335899353}
/home/teo/userdata/datasets/chopin_cleaned/Beethoven/Piano_Sonatas/30-1/musicxml_cleaned.musicxml
{'tempo': 0.10430591553449631, 'vel': 0.4778155982494354, 'dev': 0.5207796692848206, 'articul': 0.3154424726963043, 'pedal': 2.034499406814575, 'tempo_delta': 0.01026962324976921, 'vel_balance': 0.19826146960258484}
{'tempo': 0.04718388244509697, 'vel': 0.6076887249946594, 'dev': 0.3776440918445587, 'articul': 0.5222761631011963, 'pedal': 3.6716413497924805, 'tempo_delta': 0.010754154995083809, 'vel_balance': 0.2162013202905655}
{'tempo': 0.08158677816390991, 'vel': 0.6429473757743835, 'dev': 0.21067464351654053, 'articul': 0.4794536232948303, 'pedal': 2.489225387573242, 'tempo_delta': 0.011570528149604797, 'vel_balance': 0.2113996148109436}
{'tempo': 0.23177607357501984, 've

{'tempo': 0.12234843522310257, 'vel': 0.41979461908340454, 'dev': 1.3854281902313232, 'articul': 0.602487325668335, 'pedal': 0.852817714214325, 'tempo_delta': 0.06993080675601959, 'vel_balance': 0.1854824423789978}
{'tempo': 0.15648269653320312, 'vel': 0.41744908690452576, 'dev': 1.177114486694336, 'articul': 0.5534542202949524, 'pedal': 0.9594690203666687, 'tempo_delta': 0.08667147904634476, 'vel_balance': 0.15377922356128693}
{'tempo': 0.182728573679924, 'vel': 0.3847889304161072, 'dev': 3.3555023670196533, 'articul': 1.1375253200531006, 'pedal': 0.7999278903007507, 'tempo_delta': 0.093482606112957, 'vel_balance': 0.16490302979946136}
{'tempo': 0.2390735149383545, 'vel': 0.68206387758255, 'dev': 3.689852714538574, 'articul': 1.0084280967712402, 'pedal': 0.8084574937820435, 'tempo_delta': 0.10715856403112411, 'vel_balance': 0.22394908964633942}
{'tempo': 0.16625423729419708, 'vel': 0.6038715839385986, 'dev': 1.2494113445281982, 'articul': 0.7152304649353027, 'pedal': 0.872375369071960

{'tempo': 0.019339514896273613, 'vel': 0.8717755079269409, 'dev': 0.20290935039520264, 'articul': 0.7347903847694397, 'pedal': 0.771590530872345, 'tempo_delta': 0.006611206568777561, 'vel_balance': 0.1949717253446579}
/home/teo/userdata/datasets/chopin_cleaned/Liszt/Concert_Etude_S145/2/musicxml_cleaned.musicxml
{'tempo': 0.29354530572891235, 'vel': 1.1960382461547852, 'dev': 2.5385189056396484, 'articul': 0.6134502291679382, 'pedal': 0.8997816443443298, 'tempo_delta': 0.05988283082842827, 'vel_balance': 0.5013801455497742}
{'tempo': 0.1752547025680542, 'vel': 1.0057936906814575, 'dev': 2.453251838684082, 'articul': 0.9014197587966919, 'pedal': 0.7939301133155823, 'tempo_delta': 0.04079011082649231, 'vel_balance': 0.5803394913673401}
{'tempo': 0.3519546389579773, 'vel': 0.8067633509635925, 'dev': 1.862430453300476, 'articul': 0.8265467286109924, 'pedal': 1.016870141029358, 'tempo_delta': 0.07214868813753128, 'vel_balance': 0.4714335501194}
/home/teo/userdata/datasets/chopin_cleaned/Lis

{'tempo': 0.3413151502609253, 'vel': 0.46638303995132446, 'dev': 0.510757327079773, 'articul': 0.8808479905128479, 'pedal': 0.9405127167701721, 'tempo_delta': 0.02857980690896511, 'vel_balance': 0.4632909297943115}
{'tempo': 0.22225576639175415, 'vel': 0.5429410338401794, 'dev': 0.47041091322898865, 'articul': 1.0002695322036743, 'pedal': 0.8891189098358154, 'tempo_delta': 0.023628966882824898, 'vel_balance': 0.540974497795105}
{'tempo': 0.039477646350860596, 'vel': 0.4713534116744995, 'dev': 0.2958658039569855, 'articul': 0.6285828948020935, 'pedal': 0.9200928211212158, 'tempo_delta': 0.012932660058140755, 'vel_balance': 0.4712163507938385}


In [81]:
args.vel_balance_loss

False